In [20]:
import pandas as pd
import numpy as np

data_noise_hourly = pd.read_csv("../Data/hourly_noisedata_2022.csv")
data_Namen = data_noise_hourly[data_noise_hourly["description"]=="MP 03: Naamsestraat 62 Taste"]
data_noise_hourly.head()


,month,day,hour,description,lamax,laeq
0,1,1,0,MP 03: Naamsestraat 62 Taste,60.322528,57.126833
1,1,1,0,MP 05: Calvariekapel KU Leuven,53.230972,49.987639
2,1,1,0,MP 06: Parkstraat 2 La Filosovia,53.666056,50.752000
3,1,1,0,MP 07: Naamsestraat 81,50.056861,47.440222
4,1,1,1,MP 03: Naamsestraat 62 Taste,53.033583,50.853806


In [12]:
#detect missing datapoint in data_Namen

missing_hours = []
days_month = [31,28,31,30,31,30,31,31,30,31,30,31]

# Iterate over each day of the year
for month in range(1, 13):
    for day in range(1, days_month[month-1]):  # Assuming maximum day is 31 for simplicity
        # Filter the data for the specific month and day
        day_data = data_Namen[(data_Namen["month"] == month) & (data_Namen["day"] == day)]

        # Get the list of hours present in the data for the current day
        hours = day_data["hour"].tolist()

        # Check if there is any missing hour in the sequence
        for hour in range(24):
            if hour not in hours:
                missing_hours.append((month, day, hour))

# Print the missing hour(s), if any
if missing_hours:
    print("Missing hour(s) detected:")
    for month, day, hour in missing_hours:
        print(f"Month: {month}, Day: {day}, Hour: {hour}")
else:
    print("No missing hours found.")

Missing hour(s) detected:
Month: 3, Day: 27, Hour: 2


In [16]:
#different locations

# Extract the unique locations from the "description" column
locations = sorted(data_noise_hourly["description"].unique())
#sorted to get in alphabetical order

# Print the unique locations
print("Unique locations:")
for location in locations:
    print(location)

Unique locations:
MP 01: Naamsestraat 35  Maxim
MP 02: Naamsestraat 57 Xior
MP 03: Naamsestraat 62 Taste
MP 04: His & Hears
MP 05: Calvariekapel KU Leuven
MP 06: Parkstraat 2 La Filosovia
MP 07: Naamsestraat 81
MP08bis - Vrijthof


In [23]:
number_locations = len(locations)
print(number_locations)

angle = 360/number_locations
print(angle)

# Create an array from 0 to 360 with steps of 45
angles = np.arange(0, 360, angle) #360 not included

# Print the array
print(angles)

8
45.0
[  0.  45.  90. 135. 180. 225. 270. 315.]


In [74]:
# Compute the average "laeq" for each location
average_laeq = data_noise_hourly.groupby("description")["laeq"].mean()
average_lamax = data_noise_hourly.groupby("description")["lamax"].mean()

# Print the average "laeq" for each location
print("Average laeq for each location:")
print(average_laeq)

# Create a new dataframe by combining the average values
average_noise_per_location = pd.DataFrame({'Average laeq': average_laeq, 'Average lamax': average_lamax})

# Add the "description" as a column
average_noise_per_location['description'] = average_noise_per_location.index

# Remove the portion before ":" or "-" symbol, including any preceding numbers
average_noise_per_location['description'] = average_noise_per_location['description'].str.replace('.*?(?!(\d|$))[:|-]', '', regex=True)
#using a negative lookhead to keep the string if "\d" removes the complete string

average_noise_per_location['description'] = average_noise_per_location['description'].str.replace('.*?(?!\d{2,}$)(\d{1,2})', '', regex=True)



# Remove the portion before the digit or two digits, including the digit(s), except if it removes the complete string
#average_noise_per_location['description'] = average_noise_per_location['description'].apply(lambda x: np.nan if x == x.replace(x, '') else x.replace('.*?(?!\d{2,}$)(\d{1,2})', '', regex=True))

# Reset the index
average_noise_per_location.reset_index(drop=True, inplace=True)

# Remove the text "KU Leuven" if it is present
average_noise_per_location['description'] = average_noise_per_location['description'].str.replace('KU Leuven', '', regex=False)
#exception that I cannot fix with code
average_noise_per_location.loc[6, 'description'] = "Naamsestraat 81"


# Print the combined dataframe
print(average_noise_per_location)

print(average_noise_per_location.columns)

Average laeq for each location:
description
MP 01: Naamsestraat 35  Maxim       52.371057
MP 02: Naamsestraat 57 Xior         51.245595
MP 03: Naamsestraat 62 Taste        53.014597
MP 04: His & Hears                  53.792969
MP 05: Calvariekapel KU Leuven      50.704132
MP 06: Parkstraat 2 La Filosovia    50.799555
MP 07: Naamsestraat 81              49.720100
MP08bis - Vrijthof                  44.991085
Name: laeq, dtype: float64
   Average laeq  Average lamax      description
0     52.371057      54.672615            Maxim
1     51.245595      53.238218             Xior
2     53.014597      54.739970            Taste
3     53.792969      55.507523      His & Hears
4     50.704132      52.789002   Calvariekapel 
5     50.799555      52.747400     La Filosovia
6     49.720100      51.350269  Naamsestraat 81
7     44.991085      46.527490         Vrijthof
Index(['Average laeq', 'Average lamax', 'description'], dtype='object')


In [82]:
import plotly.graph_objects as go

fig = go.Figure(go.Barpolar(
    r=average_noise_per_location["radius"],
    theta=angles,
    width=average_noise_per_location["angle"],
    marker_color=["#E4FF87", '#709BFF',  '#FFDF70', '#FFAA70', '#96D5B3', '#FFC2A0', '#B6FFB4', '#FFDEA6'],
    marker_line_color="black",
    marker_line_width=2,
    opacity=0.8
))

fig.update_layout(
    template=None,
    margin=dict(l=0, r=0, t=20, b=20),  # Set margins to 0
    polar = dict(
        radialaxis = dict(range=[0, 5], showticklabels=False, ticks=''),
        angularaxis=dict(
            tickmode='array',
            tickvals=angles,
            ticktext=average_noise_per_location["description"],
            showticklabels=True,
            ticks=''
        )
    )
)

fig.show()